In [1]:
import numpy as np

In [2]:
def parse_input(filename: str):
    elf_lst = []
    with open(filename) as f:
        for row_idx, row in enumerate(f):
            row = row.rstrip()
            for col_idx, char in enumerate(row):
                if char == "#":
                    elf_lst.append(col_idx + row_idx * 1j)

    return np.array(elf_lst)

In [3]:
from tqdm import tqdm, trange

diffs = np.stack(
    [
        -1 - 1j,
        -1j,
        1 - 1j,
        1,
        1 + 1j,
        1j,
        -1 + 1j,
        -1,
        -1 - 1j,
    ]
)


def print_field(elf_pos: np.ndarray):
    elf_pos_set = set(map(tuple, elf_pos.view('(2,)float')))
    ret_str = ""
    for row_idx in range(elf_pos[:, 1].min(), elf_pos[:, 1].max() + 1):
        for col_idx in range(elf_pos[:, 0].min(), elf_pos[:, 0].max() + 1):
            if (col_idx, row_idx) in elf_pos_set:
                ret_str += "#"
            else:
                ret_str += "."
        ret_str += "\n"
    print(ret_str)


def task_one(elf_pos: set[np.ndarray], num_rounds: int):
    dirs = np.array([0, 2, 3, 1], dtype=int)

    new_elfs = diffs[:, None] + elf_pos

    # for all possible movements for all pair of elfs
    # does the movement of one elf hit the other
    full_mask = new_elfs[:, :, None] == elf_pos

    for _round in trange(num_rounds):
        mask = full_mask.any(-1)
        has_neighs = mask.any(0)

        if not has_neighs.any():
            break

        curr_mask = has_neighs

        proposed_moves = []
        proposing_elf_inds = []

        for dir in dirs:
            dir_possible = ~mask[2 * dir : 2 * dir + 3].any(0) & curr_mask
            curr_mask = curr_mask & ~dir_possible
            proposed_moves.append(new_elfs[2 * dir + 1, dir_possible])
            proposing_elf_inds.append(np.where(dir_possible)[0])

        proposed_moves = np.concatenate(proposed_moves, 0)
        proposing_elf_inds = np.concatenate(proposing_elf_inds, 0)

        if not proposed_moves.size:
            break

        unique_pos, unique_inds, unique_counts = np.unique(
            proposed_moves,
            return_index=True,
            return_counts=True,
            axis=0,
        )
        update_inds = unique_inds[np.where(unique_counts == 1)]
        new_move_inds = proposing_elf_inds[update_inds]

        if not new_move_inds.size:
            break

        elf_pos[new_move_inds] = proposed_moves[update_inds]
        new_elfs[:, new_move_inds] = diffs[:, None] + elf_pos[new_move_inds]

        full_mask[:, new_move_inds] = new_elfs[:, new_move_inds, None] == elf_pos
        full_mask[:, :, new_move_inds] = new_elfs[:, :, None] == elf_pos[new_move_inds]

        dirs = np.roll(dirs, -1)

    return (
        np.prod(
            elf_pos.view("(2,)float").max(0)
            - elf_pos.view("(2,)float").min(0)
            + np.ones(2)
        )
        - len(elf_pos),
        _round + 1,
    )

In [4]:
elfs = parse_input("test-input.txt")
t1_res = task_one(elfs, 10)
assert (110, 10) == t1_res, f"{t1_res} != {(110, 10)}"

100%|██████████| 10/10 [00:00<00:00, 1586.11it/s]


In [5]:
elfs = parse_input("input.txt")
assert (4288, 10) == task_one(elfs, 10)

100%|██████████| 10/10 [00:02<00:00,  3.93it/s]


In [6]:
elfs = parse_input("test-input.txt")
assert (146, 20) == task_one(elfs, 10000000)

  0%|          | 19/10000000 [00:00<1:01:27, 2711.90it/s]


In [27]:
elfs = parse_input("input.txt")
task_one(elfs, 10000000)

  0%|          | 939/10000000 [06:49<1212:19:55,  2.29it/s]


(16496.0, 940)